###### Example of a file saved in the NXarpes format

To help the development of community standard and software interface, here is also the example of a script to dynamically build a NeXus file formatted according to the NXarpes application definition as developed by P.R. Jemian and T. Richter (contributors on GitHub, but my feeling is that also the photoemission community in DIAMOND, and in particular M. Hoesch and P. Dudin were involved).

Here, I do not try to save *exclusively* the metadata required by the definition, but to save a meaningful. The required ones are anyway separated. 

In [1]:
import h5py
import numpy as np
import os
import six
import pytest

from nexusformat.nexus import *


def printname(name):
    print(name)

# Read data from hdf5 file

In [2]:
#Reading the volumetric binned data from hdf5 file
in_d = h5py.File('Original Files/201805_WSe2_LCPpump_LPprobe.h5', 'r')
#Show hdf5 hierarchy (dictionary-like syntax)
in_d.visit(printname) # May have large output

axes
axes/E
axes/kx
axes/ky
axes/tpp
binned
binned/V


In [3]:
#NXarpes is designed to accept data from analysers in the shape of stacks of 2D images. I reduce the format
#using slicing to match that.

low_3D_data=in_d['binned']['V'][:,38:42,:,:].sum(axis=(1))

# Create NXentry object

General entry object. An entry relates to a single measurement. Parameters directly at this level contain very general information to contextualize the experiment such as proposal title, beamtime ID, et similia.

In [4]:
root=NXroot(NXentry())

In [5]:
# Fill general parameters of NXentry
root.entry.title=NXfield('Excited-state dynamics of WSe2 in the Valence Band and Core-Levels')
root.entry.start_time=NXfield('2018-05-01T07:22:00+02:00')
root.entry.definition=NXfield('NXarpes')

# The following entries are not required
root.entry.experiment_identifier=NXfield('F-20170538')
root.entry.run_cycle=NXfield('2018 User Run Block 2')
root.entry.entry_identifier=NXfield('Run 22118')
root.entry.end_time=NXfield('2018-05-01T09:22:00+02:00')
root.entry.duration=NXfield(7200, units='s')
root.entry.collection_time=NXfield(7200, units='s')

   ## Create NXinstrument object

General "Instrument" folder. The "instrument" designation refers to the whole set-up. Each individual component is defined by subclasses of this. As parameters of this group, I added the ones that are referring to an overall performance of the set-up and not of its single components, like temporal, energetic and spatial resolution.

In [6]:
root.entry.instrument=NXinstrument()

# The following entries are not required
root.entry.instrument.name=NXfield('HEXTOF @ PG-2, Flash')
root.entry.instrument.temporal_resolution=NXfield(100, units='fs')
root.entry.instrument.spatial_resolution=NXfield(500, units='um')
root.entry.instrument.energy_resolution=NXfield(100, units='meV')

### Create source:NXsource object

The "source" group refers to the properties of the lightsource used in the experiment, NOT the properties of the beam at the sample. This group is useful in case of complex sources such as large facilities, complex laser systems or sources with complex time structure (as in this case).

We propose to use "source" to indicate the primary probe source to maximize the consistency across the community. Further sources in multiple beam experiment will be specifically named with names such as "source_pump", "source_2", etc.

In [7]:
root.entry.instrument.source=NXsource()

In [8]:
root.entry.instrument.source.name=NXfield('FLASH')
root.entry.instrument.source.type=NXfield('Free Electron Laser')
root.entry.instrument.source.probe=NXfield('x-ray')

# The following parameters are not required

root.entry.instrument.source.energy=NXfield(427, units='MeV')
root.entry.instrument.source.current=NXfield(1, units='uA')
root.entry.instrument.source.frequency=NXfield(10, units='Hz')
root.entry.instrument.source.mode=NXfield('Burst')
root.entry.instrument.source.number_of_bursts=NXfield(1)
root.entry.instrument.source.burst_length=NXfield(500, units='us')
root.entry.instrument.source.burst_distance=NXfield(199.5, units='ms')
root.entry.instrument.source.bunch_length=NXfield(100, units='fs')
root.entry.instrument.source.bunch_distance=NXfield(1, units='us')
root.entry.instrument.source.number_of_bunches=NXfield(500)
root.entry.instrument.source.top_up=NXfield(True)
root.entry.instrument.source.burst_number_start=NXfield(102644001)
root.entry.instrument.source.burst_number_end=NXfield(102680129)

A second source in a pump-probe experiment is the pump source. Creating a second group of NXsource class will not create an ambiguous hierarchy, because:
1. the class is only an attribute, what appears in the hiearachy structure is the name
2. definitions may require the exisistence of a group named "source" of "NXsource" class that is fully general and refers only to the probe source, and will not suffer ambiguity with the "pump_source"

In [9]:
root.entry.instrument.source_pump=NXsource()

In [10]:
root.entry.instrument.source_pump.name=NXfield('User Laser @ FLASH')
root.entry.instrument.source_pump.type=NXfield('OPCPA')
root.entry.instrument.source_pump.probe=NXfield('NIR')

# The following parameters are not required

root.entry.instrument.source_pump.frequency=NXfield(10, units='Hz')
root.entry.instrument.source_pump.mode=NXfield('Burst')
root.entry.instrument.source_pump.number_of_bursts=NXfield(1)
root.entry.instrument.source_pump.burst_length=NXfield(400, units='us')
root.entry.instrument.source_pump.burst_distance=NXfield(199.6, units='ms')
root.entry.instrument.source_pump.bunch_length=NXfield(50, units='fs')
root.entry.instrument.source_pump.bunch_distance=NXfield(1, units='us')
root.entry.instrument.source_pump.number_of_bunches=NXfield(400)
root.entry.instrument.source_pump.rms_jitter=NXfield(204.68816194453154, units='fs')

### Create beam_pump_0 object

I reduce, but leave the beam objects because they do not interfere with the definition (they are not required/constrained), but relay important metadata. 

In [11]:
# The following class is not required

root.entry.instrument.beam_pump_0=NXbeam()

In [12]:
# The following parameters are not required

root.entry.instrument.beam_pump_0.distance=NXfield(0, units='cm')
root.entry.instrument.beam_pump_0.pulse_energy=NXfield(15.71, units='uJ')
root.entry.instrument.beam_pump_0.average_power=NXfield(62.848, units='mW')
root.entry.instrument.beam_pump_0.photon_energy=NXfield(1.55, units='eV')
root.entry.instrument.beam_pump_0.center_wavelength=NXfield(800, units='nm')
root.entry.instrument.beam_pump_0.polarization_angle=NXfield(np.nan, units='deg') # Angle of polarization ellipse 
# from plane of incidence. For circular polarization, it is not defined.
root.entry.instrument.beam_pump_0.polarization_ellipticity=NXfield(-1.0) # Ellipticity of polarization.
# 0 for linear, +1 for RCP, -1 for LCP.
root.entry.instrument.beam_pump_0.size_x=NXfield(500, units='um')
root.entry.instrument.beam_pump_0.size_y=NXfield(200, units='um')
root.entry.instrument.beam_pump_0.fluence=NXfield(5, units='mJ/cm^2')
root.entry.instrument.beam_pump_0.pulse_duration=NXfield(50, units='fs')

### Create beam_probe_0 object

Analogous structure for the probe beam.

In [13]:
# The following class is not required

root.entry.instrument.beam_probe_0=NXbeam()

In [14]:
# The following parameters are not required

root.entry.instrument.beam_probe_0.distance=NXfield(0, units='cm')
root.entry.instrument.beam_pump_0.pulse_energy=NXfield(1.24258, units='nJ')
root.entry.instrument.beam_pump_0.average_power=NXfield(6.21289, units='uW')
root.entry.instrument.beam_probe_0.photon_energy=NXfield(36.49699020385742, units='eV')
root.entry.instrument.beam_probe_0.polarization_angle=NXfield(0.0, units='deg') # Angle of polarization ellipse 
# from plane of incidence. For circular polarization, it is not defined.
root.entry.instrument.beam_probe_0.polarization_ellipticity=NXfield(0.0) # Ellipticity of polarization.
# 0 for linear, +1 for RCP, -1 for LCP.
root.entry.instrument.beam_probe_0.size_x=NXfield(500, units='um')
root.entry.instrument.beam_probe_0.size_y=NXfield(200, units='um')
root.entry.instrument.beam_probe_0.pulse_duration=NXfield(70, units='fs')

### Create NXmonochromator object

Group for monochromator metadata, can include specific information about motor positions etc.

In [15]:
root.entry.instrument.monochromator=NXmonochromator()

# The following subclass is not required

root.entry.instrument.monochromator.slit=NXslit()

In [16]:
root.entry.instrument.monochromator.energy=NXfield(36.49699020385742, units='eV')

# The following parameters are not required

root.entry.instrument.monochromator.energy_error=NXfield(0.21867309510707855, units='eV')
root.entry.instrument.monochromator.slit.y_gap=NXfield(2000.04833984375, units='um')

### Create manipulator:NXpositioner object 

In [17]:
# The following class is not required

root.entry.instrument.manipulator=NXpositioner()

In [18]:
# The following parameters are not required

root.entry.instrument.manipulator.type=NXfield('Hexapod')
root.entry.instrument.manipulator.pos_x1=NXfield(11.3,units='um')
root.entry.instrument.manipulator.pos_x2=NXfield(11.3,units='um')
root.entry.instrument.manipulator.pos_y=NXfield(7.2,units='um')
root.entry.instrument.manipulator.pos_z1=NXfield(20.77,units='um')
root.entry.instrument.manipulator.pos_z2=NXfield(21.20,units='um')
root.entry.instrument.manipulator.pos_z3=NXfield(20.22,units='um')
root.entry.instrument.manipulator.sample_temperature=NXfield(300,units='K')
root.entry.instrument.manipulator.sample_bias=NXfield(29,units='V')

### Create NXdetector object

NXdetector is the class of objects storing the metadata relative to the analyser

In [19]:
# The following class is required and the name is constrained

root.entry.instrument.analyser=NXdetector()

In [20]:
root.entry.instrument.analyser.lens_mode=NXfield('20180430_KPEEM_M_-2.5_FoV6.2_rezAA_20ToF_focused.sav')
root.entry.instrument.analyser.acquisition_mode=NXfield('fixed') # Has constrained values ('fixed' or 'swept')

root.entry.instrument.analyser.entrance_slit_shape=NXfield('straight') # Has constrained values ('curved' or 'straight')
# NB the momentum microscope does not have slit (it has apertures). But this field is constrained this way.

root.entry.instrument.analyser.entrance_slit_setting=NXfield(0)
root.entry.instrument.analyser.entrance_slit_size=NXfield(750, units='um')
root.entry.instrument.analyser.pass_energy=NXfield(20, units='eV')
root.entry.instrument.analyser.time_per_channel=NXfield(7200, units='s')

# Follows a set of detector specific parameters. I do not know if there is software relying on NXarpes 
# definition and if it relies on the following parameters to do simple operations, so I enter logical values 
# (not from the experiment) to make the file usable.

root.entry.instrument.analyser.sensor_size=NXfield([80,146]) 
root.entry.instrument.analyser.region_origin=NXfield([0,0]) 
root.entry.instrument.analyser.region_size=NXfield([80,146]) 


# In  NXarpes the data are saved in the NXdetector group, and then linked in the NXdata one.
# I see a major issue here, restricting the use of the definition.
# It requires two axes: angles and energies. Angles is an intermediate quantity, as the final data are in
# wavenumber units. The momentum microscope does not have a reliable calibration for angles, we scle directly
# to k. Also, the simple naming: "angles" can create ambiguity even for a tilt map with an hemispherical
# where 2 angular axes are scanned.

root.entry.instrument.analyser.angles=NXfield(in_d['axes']['kx'],units='1/Å') 
root.entry.instrument.analyser.energies=NXfield(in_d['axes']['E'],name='Energy',units='eV')
root.entry.instrument.analyser.data=NXfield(low_3D_data,units='counts') 

# The following parameters are not required

root.entry.instrument.analyser.delays=NXfield(in_d['axes']['tpp'],name='Pump-probe delay',units='fs')

root.entry.instrument.analyser.extractor_voltage=NXfield(6030, units='V')
root.entry.instrument.analyser.working_distance=NXfield(4, units='mm')
root.entry.instrument.analyser.projection=NXfield('reciprocal')
root.entry.instrument.analyser.magnification=NXfield(-1.5)
root.entry.instrument.analyser.field_aperture_x=NXfield(-0.200, units='um')
root.entry.instrument.analyser.field_aperture_y=NXfield(5.350, units='um')
root.entry.instrument.analyser.contrast_aperture=NXfield('Out')
root.entry.instrument.analyser.dispersion_scheme=NXfield('Time of flight')
root.entry.instrument.analyser.amplifier_type=NXfield('MCP')
root.entry.instrument.analyser.detector_type=NXfield('DLD')
root.entry.instrument.analyser.sensor_count=NXfield(4)

## Create NXsample object

NXsample is a base class (only subordinate to entry) where information regarding the sample is stored. Note that the temperature is cross linked from the sample_temperature field in the manipulator, as an example of cross linking data from NXinstrument hierarchy in more elevated and descriptive hierarchy positions.

In [21]:
root.entry.sample=NXsample()

In [22]:
root.entry.sample.name=NXfield('WSe2')
root.entry.sample.temperature=NXlink(root.entry.instrument.manipulator.sample_temperature)

# The following parameters are not required

root.entry.sample.state=NXfield('monocrystalline solid')
root.entry.sample.purity=NXfield(0.999)
root.entry.sample.surface_orientation=NXfield('0001')
root.entry.sample.layer=NXfield('bulk')
root.entry.sample.chemical_name=NXfield('Tungsten diselenide')
root.entry.sample.chem_id_cas=NXfield('12067-46-8')
root.entry.sample.pressure=NXfield(3.27e-10, units='mbar')
root.entry.sample.thickness=NXfield(0.5, units='mm')
root.entry.sample.bias=NXlink(root.entry.instrument.manipulator.sample_bias)
root.entry.sample.growth_method=NXfield('Chemical Vapor Deposition')
root.entry.sample.preparation_method=NXfield('in-vacuum cleave')
root.entry.sample.vendor=NXfield('HQ Graphene')

## Create NXdata object

Main data group containing the data relevant for the entry. The calibrated axes on which it should be plotted are linked from the NXinstrument:NXdetector group, where they are stored.

NXfield inherits data allocation fro h5py. When data is above a cerain size, it divides it in chunks for better memory stability. It also automatically applies a lossless compression. While the chunking may be desirable in this situation, compression may be not. Changing the "compression_opts" number between 0 and 9 allows to go from, respectively, no compression to maximum compression.

In [23]:
# Create the NXdata object
kx=NXlink(root.entry.instrument.analyser.angles)
E=NXlink(root.entry.instrument.analyser.energies)
tpp=NXlink(root.entry.instrument.analyser.delays)
counts=NXlink(root.entry.instrument.analyser.data)

In [24]:
# Wrap the NXfields in a NXdata object
root.entry.data=NXdata(counts,[kx,E,tpp]) 

# Save File

The saving command wraps all the fields, groups and data with the attributes according to NeXus Standard and saves in an h5.

In [25]:
root.save('201805_WSe2_arpes.nxs', mode='w')

NXroot('root')